Get OSM segments as specified in this [link](https://www.npmjs.com/package/movement-data-toolkit)

In [1]:
from shapely.geometry import Polygon, Point, LineString
from shapely.ops import cascaded_union
from shapely import wkt
import pandas as pd
import keplergl as kp
import utm
import geopandas as gpd
from itertools import compress 

#!pip install -i https://test.pypi.org/simple/ gtfs-functions-VERSION-1
import gtfs_functions as gtfs

# GTFS

We will start only a couple of lines in the city of Miami.

In [2]:
# Extract the GTFS
gtfs_path = r"C:\Users\santi\Dropbox (Remix)\Remix\Data Team\Data Analytics\Load\GTFS\MIAMI-DADE\MIAMI.zip"

routes, stops, stop_times, trips, shapes, gtfs_list = gtfs.import_gtfs(gtfs_path, busiest_date = False)

In [3]:
rfilter = routes.loc[routes.route_short_name.isin(['9', '10']), 'route_id'].unique()

sfilter = stop_times.loc[stop_times.route_id.isin(rfilter)].reset_index()

stops_filter = stops.loc[stops.stop_id.isin(sfilter.stop_id.unique())].reset_index()
shapes_filter = shapes.loc[shapes.shape_id.isin(sfilter.shape_id.unique())].reset_index()

# Cut the segments
bsegments_gdf = gtfs.cut_gtfs(stop_times, stops_filter, shapes_filter)
bsegments_gdf.route_id.unique()

In [920]:
m = kp.KeplerGl(height=400, data=dict(data=bsegments_gdf, name='Lines'))
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':    route_id  direction_id  stop_sequence start_stop_id  \
0     22753             0    …

In [23]:
# Buffer the lines 5 meters
b_segments = bsegments_gdf.loc[:,['segment_id', 'geometry']].drop_duplicates(subset='segment_id').reset_index().drop('index', axis=1)

# Save memory changing the data types
stop_segments = pd.DataFrame()
stop_segments['stops_id'] = b_segments.segment_id
stop_segments['segment_id'] = list(range(1, len(b_segments)+1))

b_segments['segment_id'] = stop_segments['segment_id']

int_col = b_segments.select_dtypes(include=['int64'])
columns = int_col.columns
converted_int = int_col.apply(pd.to_numeric, downcast='unsigned')
b_segments = pd.merge(b_segments.drop(columns, axis=1), converted_int, left_index=True, right_index=True, how='left')

# Get a coord system in meters
lat = b_segments.geometry[0].coords[0][1]
lon = b_segments.geometry[0].coords[0][0]

zone = utm.from_latlon(lat, lon)

def code(zone):
    #The EPSG code is 32600+zone for positive latitudes and 32700+zone for negatives.
    if lat <0:
        epsg_code = 32700 + zone[2]
    else:
        epsg_code = 32600 + zone[2]
    return epsg_code

# Create the buffers
buffer_size = 10

buffers = gpd.GeoDataFrame(data = b_segments['segment_id'], geometry = b_segments.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
kp.KeplerGl(height=400, data=dict(data=buffers, name='buffers'))

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':      segment_id                                           geometry
0             1  POL…

In [6]:
# Merge buffers into one polygon to speed up the spatial join (for filtering)
polygons = [p for p in buffers.geometry]
    
u = cascaded_union(polygons)

df = pd.DataFrame({'poly': [1], 'geometry':u.wkt})
df['geometry'] = df['geometry'].apply(wkt.loads)
lines_poly = gpd.GeoDataFrame(data=df.poly, geometry = df.geometry)

# Download OSM segments

In [7]:
# Query OSM with Overpass API
# Filter only roads (no POIs or nodes)
import requests
import json

bounds = lines_poly.geometry.total_bounds

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
(way["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|residential|service|living_street"]
["access"!~"private|no"]
({0}, {1}, {2}, {3}););
out geom;
""".format(bounds[1], bounds[0], bounds[3], bounds[2])

response = requests.get(overpass_url, 
                        params={'data': overpass_query})

data = response.json()
ways_df = pd.DataFrame(data['elements'])

In [8]:
# Transform into a geo data frame with line segments from node to node
way_ids = []
oneway = []
node_ids = []
lat_lon = []
segment_seq = []

for e in data['elements']:
    way_ids = way_ids + [e['id']]*len(e['nodes'])
    oneway = oneway + [e['tags'].get('oneway', '0')]*len(e['nodes'])
    segment_seq = segment_seq + list(range(1, len(e['nodes'])+1))
    node_ids = node_ids + e['nodes']
    lat_lon = lat_lon + e['geometry']

# Convert to int to save memory
oneway = [1 if s=='yes' else s for s in oneway] 
oneway = [0 if s=='no' else s for s in oneway] 
oneway = list(map(int, oneway))

# Parse the json into a dataframe
segments = pd.DataFrame()
segments['way_id'] = way_ids
segments['oneway'] = oneway
segments['segment_seq'] = segment_seq


# Get lat,lon values right
lat = [p['lat'] for p in lat_lon]
lon = [p['lon'] for p in lat_lon]

# Define our lists
bool_list = segments['way_id'] == segments['way_id'].shift(-1)
segment_nodes = ['{0} - {1}'.format(str(node_ids[i]), str(node_ids[i+1])) for i in range(0,len(node_ids)-1)]
segment_ids = list(range(1, len(segment_nodes)+1))
points =  [Point(lon[i], lat[i]) for i in range(0, len(lat))]
points_next = points[1:] + [None]

# Remove the last node of the segment (it is already in the last segment)
segment_ids = list(compress(segment_ids, bool_list)) 
segment_nodes = list(compress(segment_nodes, bool_list)) 
points = list(compress(points, bool_list)) 
points_next = list(compress(points_next, bool_list)) 
geometry = [LineString([points[i], points_next[i]]) for i in range(0,len(segment_ids))]

# Store the nodes of each segment
nodes_seg = pd.DataFrame({'osm_segment_id': segment_ids, 'nodes': segment_nodes})

# Keep the good data and create the geo data frame
segments = segments.loc[bool_list]
segments['osm_segment_id'] = segment_ids
segments_gdf = gpd.GeoDataFrame(data=segments, geometry = geometry)

In [9]:
# Save some memory
int_col = segments_gdf.select_dtypes(include=['int64'])
columns = int_col.columns
converted_int = int_col.apply(pd.to_numeric, downcast='unsigned')

segments_gdf = pd.merge(segments_gdf.drop(columns, axis=1), converted_int, left_index=True, right_index=True, how='left')
segments_gdf.info(memory_usage="deep")

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 51408 entries, 0 to 63643
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   geometry        51408 non-null  geometry
 1   way_id          51408 non-null  uint32  
 2   oneway          51408 non-null  int64   
 3   segment_seq     51408 non-null  uint8   
 4   osm_segment_id  51408 non-null  uint16  
dtypes: geometry(1), int64(1), uint16(1), uint32(1), uint8(1)
memory usage: 4.0 MB


In [10]:
%%time
# Keep only the osm segments that intersect the bus lines
osm = gpd.sjoin(segments_gdf, lines_poly, op='intersects').reset_index().drop(['index_right', 'poly', 'index'], axis=1)

Wall time: 20.8 s


In [874]:
osm.info(memory_usage="deep")

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3102 entries, 0 to 3101
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   geometry        3102 non-null   geometry
 1   way_id          3102 non-null   uint32  
 2   oneway          3102 non-null   int64   
 3   segment_seq     3102 non-null   uint8   
 4   osm_segment_id  3102 non-null   uint16  
dtypes: geometry(1), int64(1), uint16(1), uint32(1), uint8(1)
memory usage: 69.8 KB


In [11]:
m = kp.KeplerGl(data=dict(data=osm, name='segments'))
m.add_data(data=lines_poly, name='lines_poly')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':                                                geometry     way_id  oneway  \
0     LIN…

# Join GTFS segments to OSM segments

## 1. Approach with overlay and area calculation (discarded, go to #2)

In [575]:
# Right now we filtered  OSM segments that intersect with lines_poly
# We will keep only those that more than 50% of their area falls within lines_poly
buffer_size = 5

buffers_10 = gpd.GeoDataFrame(data = b_segments['segment_id'], geometry = b_segments.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
polygons = [p for p in buffers_10.geometry]
u = cascaded_union(polygons)

df = pd.DataFrame({'poly': [1], 'geometry':u.wkt})
df['geometry'] = df['geometry'].apply(wkt.loads)
lines_poly_10 = gpd.GeoDataFrame(data=df.poly, geometry = df.geometry)


osm.crs = {'init':'epsg:4326'}
osm_bu = gpd.GeoDataFrame(data = osm.drop('geometry', axis=1), geometry = osm.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
osm_bu['area_osm'] = osm_bu.area

In [576]:
# %%time
# #osm_test = intersections.loc[intersections.segment_id=='8908-8826']
# segments_gdf.crs = {'init':'epsg:4326'}
# osm_bu = gpd.GeoDataFrame(data = segments_gdf.drop('geometry', axis=1), geometry = segments_gdf.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
# osm_bu['area_osm'] = osm_bu.area

last = gpd.overlay(osm_bu, lines_poly_10, how='intersection')
last['area_overlay'] = last.area
last['per'] = last['area_overlay']/last['area_osm']*100
last = last.loc[last.per>30]

In [578]:
len(osm), len(last)

(3102, 2144)

In [734]:
#osm = osm.loc[osm.osm_segment_id.isin(last.osm_segment_id.unique())]
m = kp.KeplerGl(height=600, data=dict(data=osm.loc[osm.osm_segment_id.isin(last.osm_segment_id.unique())], name='bus'))
m.add_data(data= lines_poly_10, name='lines')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':          way_id  segment_seq           osm_segment_id  \
2      11105776            1  …

## 2. Approach with angles and perpendicularity

The idea of this approach is to see with which bus segment each osm segment intersects and compare their angles. 
If their angles are closer to perpendicularity than parallelism, then we'll discard those segments.

For this approach we first need to divide each bus segment in shorter lines of only two points. 
The reason behind this is that their original shape is more complex than a straight line and it is not posible to calculate their slope.

In [14]:
# Split bus segments into two-points segments and calculate it's angle
# For angle is atan(deltay/deltax)
# 180 degrees = pi 

import math

def small_segments(s):
    lat = s.coords.xy[1]
    lon = s.coords.xy[0]

    ss = [LineString([(lon[i], lat[i]), (lon[i+1], lat[i+1])]) for i in range(0,len(lat)-1)]
    angle = [math.atan2((lat[i+1] - lat[i]),(lon[i+1]-lon[i]))*180/math.pi for i in range(0,len(lat)-1)]
    ss_seq = list(range(0,len(ss)))
    n = len(ss)
    
    aux_df = pd.DataFrame()
    aux_df['ss_seq'] = pd.to_numeric([int(x) for x in ss_seq], downcast='signed')
    aux_df['angle'] = pd.to_numeric([int(x) for x in angle], downcast='signed')
    aux_df['geometry'] = ss
    
    return aux_df

def n_ss(s):
    n = len(s.coords.xy[1])-1
    return n    

ss = b_segments.loc[:, 'geometry'].apply(small_segments)
ss_df = pd.concat(list(ss))

n = b_segments.loc[:, 'geometry'].apply(n_ss)

seg_id = [[b_segments.loc[i, 'segment_id']]*n[i]  for i in range(0,len(n))]
seg_id = [item for sublist in seg_id for item in sublist]

ss_df['segment_id'] = pd.to_numeric([x for x in seg_id], downcast='signed')
ss_df['ss_id'] = list(range(1, len(seg_id)+1)) #ss_df['segment_id'].map(str) + '-' + ss_df['ss_seq'].map(str)
ss_gdf = gpd.GeoDataFrame(data=ss_df.drop('geometry', axis=1), geometry=ss_df.geometry)
ss_gdf.head()

,ss_seq,angle,segment_id,ss_id,geometry
0,0,2,1,1,"LINESTRING (-80.17219 25.92927, -80.17111 25.9..."
1,1,2,1,2,"LINESTRING (-80.17111 25.92932, -80.17067 25.9..."
2,2,-3,1,3,"LINESTRING (-80.17067 25.92934, -80.17054 25.9..."
3,3,4,1,4,"LINESTRING (-80.17054 25.92933, -80.17017 25.9..."
4,4,-93,1,5,"LINESTRING (-80.17017 25.92936, -80.17019 25.9..."


In [25]:
# Buffer the small bus segments
buffer_size = 7

ss_gdf.crs = {'init':'epsg:4326'}
ss_buffers = gpd.GeoDataFrame(data = ss_gdf.drop('geometry', axis=1), geometry = ss_gdf.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
ss_buffers.head()

,ss_seq,angle,segment_id,ss_id,geometry
0,0,2,1,1,"POLYGON ((-80.17112 25.92938, -80.17111 25.929..."
1,1,2,1,2,"POLYGON ((-80.17067 25.92940, -80.17067 25.929..."
2,2,-3,1,3,"POLYGON ((-80.17054 25.92940, -80.17053 25.929..."
3,3,4,1,4,"POLYGON ((-80.17018 25.92942, -80.17017 25.929..."
4,4,-93,1,5,"POLYGON ((-80.17012 25.92914, -80.17012 25.929..."


In [26]:
# Calculate angle of osm segments
def angle(s):
    lat = s.coords.xy[1]
    lon = s.coords.xy[0]
    
    angle = math.atan2((lat[1] - lat[0]),(lon[1]-lon[0]))*180/math.pi
    
    return angle

osm['osm_angle'] =  pd.to_numeric(osm.geometry.apply(angle).map(int), downcast='signed')

In [27]:
%%time
ss_intersections = gpd.sjoin(osm, ss_buffers, op='intersects').reset_index()
ss_intersections['diff'] =ss_intersections['angle']-ss_intersections['osm_angle']

# Keep the segments that have "similar" angles to their intersections
# We consider "similar" angles those that have similar values or almost 180 degrees different
# in the case that osm segment doesn't have a tag "oneway=yes"
diff = 40

parallel = ss_intersections.loc[(ss_intersections['diff'].map(abs)<diff)|(ss_intersections['diff'].map(abs)>(360-diff))]
oneway_seg = ss_intersections.loc[(ss_intersections['oneway']==0)&(ss_intersections['diff'].map(abs).between(180-diff,180+diff))]
osm_filtered = parallel.append(oneway_seg)

Wall time: 707 ms


In [33]:
m = kp.KeplerGl(height=600, data=dict(data=osm_filtered, name='bus'), #config=config
               )
m.add_data(data= ss_buffers, name='lines')
#m.add_data(data= ss_buffers.loc[~ss_buffers.ss_id.isin(osm_filtered.ss_id.unique())], name='matched lines')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':        index                                           geometry     way_id  \
12       …

In [481]:
# intersections = gpd.sjoin(osm, buffers, op='intersects').reset_index()

### 2.1 Add the overlay approach

We still need to clean our OSM segments further. To do so, we will now apply the overlay approach.

This approach overlays two polygons and calculated the overlapping area. Then we match the segments with the biggest overlapping area.

To do this approach we need to buffer both, OSM segments and GTFS segments.

In [35]:
ss_buffers.head()

,ss_seq,angle,segment_id,ss_id,geometry
0,0,2,1,1,"POLYGON ((-80.17112 25.92938, -80.17111 25.929..."
1,1,2,1,2,"POLYGON ((-80.17067 25.92940, -80.17067 25.929..."
2,2,-3,1,3,"POLYGON ((-80.17054 25.92940, -80.17053 25.929..."
3,3,4,1,4,"POLYGON ((-80.17018 25.92942, -80.17017 25.929..."
4,4,-93,1,5,"POLYGON ((-80.17012 25.92914, -80.17012 25.929..."


In [37]:
buffer_size = 5

buffers_10 = gpd.GeoDataFrame(data = b_segments['segment_id'], geometry = b_segments.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
polygons = [p for p in buffers_10.geometry]
u = cascaded_union(polygons)

df = pd.DataFrame({'poly': [1], 'geometry':u.wkt})
df['geometry'] = df['geometry'].apply(wkt.loads)
lines_poly_10 = gpd.GeoDataFrame(data=df.poly, geometry = df.geometry)

osm_filtered.crs = {'init':'epsg:4326'}
osm_bu = gpd.GeoDataFrame(data = osm_filtered.drop('geometry', axis=1), geometry = osm_filtered.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
osm_bu['area_osm'] = osm_bu.area

poly_over = gpd.overlay(osm_bu.drop(['ss_seq', 'angle', 'segment_id', 'ss_id'], axis=1), lines_poly_10, how='intersection')
poly_over['area_overlay'] = poly_over.area
poly_over['per'] = poly_over['area_overlay']/poly_over['area_osm']*100
poly_over.head(1)

,index,way_id,oneway,segment_seq,osm_segment_id,osm_angle,index_right,diff,area_osm,poly,geometry,area_overlay,per
0,134,11128934,0,1,3305,1,3,1,8.195860e-08,1,"POLYGON ((-80.19784 25.88980, -80.19783 25.889...",7.554216e-08,92.171118


In [1020]:
# max_int = poly_over.pivot_table('per', index=['ss_id'], aggfunc='max').reset_index()
# max_int.rename(columns=dict(per='max_per'), inplace=True)
# poly_over = pd.merge(poly_over, max_int, how='left')
# poly_over = poly_over.loc[poly_over.per==poly_over.max_per]

39716 segments overlay
Reduced to 19816


In [39]:
m = kp.KeplerGl(height=600, data=dict(data=osm_filtered.loc[osm_filtered.osm_segment_id.isin(poly_over.osm_segment_id.unique())]), name='over',
               #config=config
               )
m.add_data(data= ss_gdf, name='lines')
m.add_data(data= poly_over, name='overlay')
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':        index                                           geometry     way_id  \
12       …

# Disregard below this for now

In [388]:
tlo = b_segments.loc[279, 'geometry'].coords.xy[0]
tla = b_segments.loc[279, 'geometry'].coords.xy[1]

tpo = [Point(tlo[i], tla[i]) for i in range(0, len(tlo))]

tli = [LineString([tpo[i], tpo[i+1]]) for i in range(0,len(tpo)-1)]

In [442]:
t_ = list(range(0,len(tli)))
t = pd.DataFrame()
t['ss_id'] = t_
t['segment_id'] = ['8908-8826']*len(tli)
t = gpd.GeoDataFrame(data=t, geometry=tli)

In [461]:
buffer_size = 3

t.crs = {'init':'epsg:4326'}

tbu = gpd.GeoDataFrame(data = t.drop('geometry', axis=1), geometry = t.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
tbu

,ss_id,segment_id,geometry
0,0,8908-8826,"POLYGON ((-80.19704 25.88977, -80.19705 25.889..."
1,1,8908-8826,"POLYGON ((-80.19782 25.88974, -80.19782 25.889..."
2,2,8908-8826,"POLYGON ((-80.19865 25.88970, -80.19866 25.889..."
3,3,8908-8826,"POLYGON ((-80.19861 25.88955, -80.19861 25.889..."
4,4,8908-8826,"POLYGON ((-80.19860 25.88902, -80.19860 25.889..."
5,5,8908-8826,"POLYGON ((-80.19863 25.88899, -80.19863 25.888..."
6,6,8908-8826,"POLYGON ((-80.19858 25.88834, -80.19858 25.888..."


In [457]:
#osm_test = intersections.loc[intersections.segment_id=='8908-8826']
osm.crs = {'init':'epsg:4326'}
osm_bu = gpd.GeoDataFrame(data = osm.drop('geometry', axis=1), geometry = osm.to_crs(epsg=code(zone)).buffer(buffer_size).to_crs(epsg=4326))
osm_bu['area_osm'] = osm_bu.area
osm_bu

,way_id,segment_seq,osm_segment_id,geometry,area_osm
0,11103228,7,98975025 - 98975028,"POLYGON ((-80.19708 25.88978, -80.19708 25.889...",4.530003e-08
1,11103228,8,98975028 - 98975031,"POLYGON ((-80.19717 25.89181, -80.19717 25.891...",1.242749e-07
2,11105776,1,98999741 - 2915146657,"POLYGON ((-80.14509 25.96005, -80.14509 25.960...",1.277068e-08
3,11106364,1,6366190747 - 6366190722,"POLYGON ((-80.14519 25.95936, -80.14519 25.959...",8.363258e-09
4,11107093,1,99010535 - 99010537,"POLYGON ((-80.19108 25.83185, -80.19108 25.831...",6.252647e-08
...,...,...,...,...,...
2759,835882554,2,7187981992 - 6945559259,"POLYGON ((-80.17005 25.92674, -80.17005 25.926...",2.200858e-08
2760,835882554,3,6945559259 - 99064510,"POLYGON ((-80.17004 25.92663, -80.17004 25.926...",9.428639e-09
2761,835882555,1,99189930 - 7802016538,"POLYGON ((-80.17007 25.92719, -80.17007 25.927...",2.043142e-08
2762,835882556,1,7802016539 - 6945095202,"POLYGON ((-80.17000 25.92584, -80.17000 25.925...",2.486940e-08
